In [ ]:
import os
from langchain_community.document_loaders import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma


os.makedirs("data", exist_ok=True)
os.makedirs("chroma_db", exist_ok=True)

csv_path = "data/places.csv"


if not os.path.exists(csv_path):
    raise FileNotFoundError(f" Could not find {csv_path}. Please add your places.csv file in the /data folder.")

loader = CSVLoader(csv_path)
docs = loader.load()
print(f" Loaded {len(docs)} rows from {csv_path}")


splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
chunks = splitter.split_documents(docs)
print(f" Split into {len(chunks)} chunks")

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = Chroma.from_documents(chunks, embeddings, persist_directory="chroma_db")
db.persist()

print(" RAG Index Built! 1000+ real Indian places loaded.")


📄 Loaded 325 rows from data/places.csv
✂️ Split into 325 chunks


c:\Users\rishi\.conda\envs\ai_project106\lib\site-packages\transformers\models\bert\modeling_bert.py:413: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


✅ RAG Index Built! 1000+ real Indian places loaded.


C:\Users\rishi\AppData\Local\Temp\ipykernel_34148\3928493766.py:30: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import re
import json
import requests
import os

app = FastAPI()

print("Loading RAG...")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = Chroma(persist_directory="chroma_db", embedding_function=embeddings)

OLLAMA_URL = "http://localhost:11434/api/generate"

class TripRequest(BaseModel):
    preferences: list[str]
    duration: int
    budget: float
    start_city: str = "Delhi"

def query_ollama(prompt: str) -> str:
    payload = {
        "model": "phi3:mini",
        "prompt": prompt,
        "stream": False,
        "options": {"temperature": 0.7}
    }
    response = requests.post(OLLAMA_URL, json=payload)
    return response.json()["response"]

def parse_itinerary(text: str):
    pattern = r"Day (\d+): (.+?) - (.+?)\. Stay: (.+?) - ₹([\d,]+)\. Food: (.+?) - ₹([\d,]+)\. Cost: ₹([\d,]+)"
    matches = re.findall(pattern, text)
    itinerary = []
    total = 0
    for m in matches:
        day, city, dest, stay, stay_cost, food, food_cost, day_cost = m
        cost = int(day_cost.replace(",", ""))
        total += cost
        itinerary.append({
            "day": int(day),
            "city": city.strip(),
            "destinations": [d.strip() for d in dest.split(",")],
            "stay": f"{stay.strip()} - ₹{stay_cost}",
            "food": f"{food.strip()} - ₹{food_cost}",
            "cost": cost
        })
    return itinerary, total

@app.post("/plan")
def plan_trip(req: TripRequest):

    query = f"{', '.join(req.preferences)} near {req.start_city} budget {req.budget}"
    docs = db.similarity_search(query, k=6)
    context = "\n".join([
        f"- {d.metadata.get('Place', 'Unknown')} ({d.metadata.get('City', '')}): {d.metadata.get('Category', '')}, ₹{d.metadata.get('Entry Fee', '500')}"
        for d in docs
    ])


    prompt = f"""
You are Touristique, India's AI travel planner.

User wants: {', '.join(req.preferences)}
Duration: {req.duration} days
Budget: ₹{req.budget:,}
Start: {req.start_city}

REAL PLACES:
{context}

Generate a {req.duration}-day itinerary:
- Use only places above
- 1-2 destinations per day
- Eco-friendly stay
- Local food
- Stay under budget

Format:
Day 1: Jaipur - Amber Fort. Stay: Eco Homestay - ₹2,800. Food: Dal Baati - ₹600. Cost: ₹5,400
""".strip()

   
    print("Calling Phi-3 via Ollama...")
    answer = query_ollama(prompt)

    try:
        itinerary, total = parse_itinerary(answer)
        return {
            "itinerary": itinerary,
            "total_cost": total,
            "summary": f"{req.duration}-day trip under ₹{req.budget:,}"
        }
    except:
        return {"raw_output": answer, "retrieved_places": context}

Loading RAG...
